In [550]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.8 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [825]:
import pandas as pd
import re
import spacy
from spacy import *

In [826]:
data = pd.read_excel("jeu_données.xlsx")
data = data.drop(['Code erreur', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
condition = (data['SYNTAXE'] == 1.0)
#data = data[condition]
data = data.drop(['LANGUE', 'SYNTAXE'], axis=1)
data = data.reset_index(drop=True)
data

,PHRASE,NOM VILLE
0,je voudrais aller de Toulouse à bordeaux,"Toulouse,Bordeaux"
1,Comment me rendre à Port-Boulet depuis la gare...,"Tours,Port-Boulet"
2,Je veux aller voir mon ami Albert à Tours en p...,"Bordeaux,Tours"
3,Il y a-t-il des trains de Nantes à Montaigu,"Nantes,Montaigu"
4,Une phrase sans origine ni destination,NOT_TRIP
5,"Si pas de numéro de séquence, on considère que...",NOT_TRIP
6,is there any train going from Paris to Marseil...,NOT_FRENCH
7,Je prévois de partir de Paris pour me rendre à...,"Paris,Nice"
8,Paris Lyon stp et fissa,"Paris, Lyon"
9,Wallah je suis pas fr,NOT_TRIP


In [827]:
titles = [_ for _ in data.loc[lambda d: d['PHRASE'].str.lower().str.contains("paris")]['PHRASE']]

In [828]:
nlp = spacy.load("fr_core_news_sm", disable=["ner"])

In [966]:
for i in range(0, len(data)):
    doc = nlp(data["PHRASE"][i])
    displacy.render(doc)

In [667]:
df = pd.read_csv("villes_france.csv", sep=',')
villes = list(df['ozan'])

/var/folders/30/j7pdftkd2nn50dll4_jtl7nr0000gn/T/ipykernel_13294/3062799863.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("villes_france.csv", sep=',')


In [668]:
sncf = pd.read_excel("timetables.xlsx")

trajet = list(sncf['trajet'])
gares = []

for k in range(0, len(trajet)):
    if len(trajet[k].split(' - ')) == 2:
        if ('Gare de ' in trajet[k].split(' - ')[0] and 'Gare de ' in trajet[k].split(' - ')[1]):
            first = trajet[k].split(' - ')[0].split('Gare de ')[1]
            last = trajet[k].split(' - ')[1].split('Gare de ')[1]
    elif len(trajet[k].split(' - ')) == 3:
        if ('Gare de ' in trajet[k].split(' - ')[0] and 'Gare de ' in trajet[k].split(' - ')[1]):
            first = trajet[k].split(' - ')[0].split('Gare de ')[1]
            last = trajet[k].split(' - ')[2].split('Gare de ')[1]
    gares.append(first)
    gares.append(last)

In [691]:
villes = [element.lower() for element in villes]
villes = list(set(villes))
villes

['villebernier',
 'blemerey-54',
 'melicocq',
 'antran',
 'lorrez-le-bocage-preaux',
 'bazainville',
 'huisnes-sur-mer',
 'hendecourt-les-ransart',
 'fontette',
 'cauzac',
 'bonsmoulins',
 'beceleuf',
 'saint-gerand',
 'saint-andre-et-appelles',
 'notre-dame-d-oe',
 'saint-quentin-la-poterie',
 'chevroliere',
 'neuville-coppegueule',
 'charavines',
 'bourdet',
 'estenos',
 'gast',
 'jonchery-sur-suippe',
 'clavieres',
 'vexaincourt',
 'dombrot-le-sec',
 'sarp',
 'etais-la-sauvin',
 'houvin-houvigneul',
 'rosieres-en-santerre',
 'croisy',
 'melicourt',
 'grattery',
 'sens-sur-seille',
 'buzancy-08',
 'bouniagues',
 'saint-jouin-de-marnes',
 'meyrieu-les-etangs',
 'ivry-la-bataille',
 'saint-jean-de-barrou',
 'cherac',
 'rosieres-43',
 'saint-benoît-01',
 'siran-34',
 'blargies',
 'mosnay',
 'saint-mesmin-24',
 'm\x9coernach',
 'sant-antonino',
 'maison-maugis',
 'chomerac',
 'bosse-60',
 'courry',
 'ruhans',
 'chateaudouble-83',
 'baconniere',
 'saint-cirq-82',
 'mesnil-jourdain',
 'neu

In [669]:
villes_gares = villes+gares
villes_gares = [element.lower() for element in villes_gares]
villes_gares = list(set(villes_gares))

In [670]:
all = list(data['PHRASE'])
detect = []
non_detect = []

def has_paris(doc):
    for i in range(0, len(doc)):
        if (doc[i].lower_ in villes_gares):
            if ((doc[i].pos_ == "PROPN" or doc[i].pos_ == "NOUN")):
                if (doc[i-1].pos_ == "ADP"):
                    return True
    return False



for doc in nlp.pipe(list(data['PHRASE'])):
    if has_paris(doc):
        print(doc)

for doc in nlp.pipe(list(data['PHRASE'])):
    if has_paris(doc):
        detect.append(doc)
    else:
        non_detect.append(doc)

je voudrais aller de Toulouse à bordeaux
Comment me rendre à Port-Boulet depuis la gare de Tours ?
Je veux aller voir mon ami Albert à Tours en partant de Bordeaux
Il y a-t-il des trains de Nantes à Montaigu
Je prévois de partir de Paris pour me rendre à Nice en vacances
Pourrais-je avoir un trajet allant de Marseille à Armentières je vous pries?
Je prévois de partir de Paris pour me rendre à Nice en vacances.
Mon ami cherche à voyager de Bordeaux à Strasbourg en train
J'ai l'intention de faire un road trip de Marseille à Lyon avec ma famille
Elle rêve de prendre l'avion de Toulouse à Montpellier pour une escapade ensoleillée
Mon collègue envisage de prendre le bus de Lille à Nantes pour une réunion
Je vais partir de Paris pour me rendre à Bordeaux en train
Mon ami envisage de voyager de Marseille à Lyon en voiture
Nous prévoyons un vol de Nice à Toulouse pour nos vacances
Elle souhaite prendre le bus de Lille à Nantes pour visiter des amis
Mon collègue cherche à organiser un trajet de

In [671]:
print(len(all), len(detect), len(non_detect))

51 38 13


In [672]:
detectable = detect
non_detectable = non_detect

In [673]:
model_name = "fr_core_news_sm"
model = spacy.load(model_name, disable=["ner"])




correct = sum(has_paris(doc) for doc in model.pipe(detectable))
wrong = sum(has_paris(doc) for doc in model.pipe(non_detectable))
precision = correct/(correct + wrong)
recall = correct/len(detectable)
accuracy = (correct + len(non_detectable) - wrong)/(len(detectable) + len(non_detectable))

f"{precision},{recall},{accuracy},{model_name}" # this is logged

'1.0,1.0,1.0,fr_core_news_sm'

In [830]:
import pandas as pd

df = pd.read_excel('jeu_données.xlsx', sheet_name='Termes')
df

,Departure words,Arrival words
0,de,à
1,depuis la,me rendre à
2,en partant de,vers
3,partir de,à la
4,allant de,vers la
5,voyager de,pour
6,depuis,vers
7,en partant de,jusqu'à
8,depuis la ville de,jusqu'à la
9,de la,mettre le cap sur


In [831]:
import numpy as np

In [871]:
arrival = df['Arrival words']
arrival = list(arrival)
arrival = [element.lower() for element in arrival if not (isinstance(element, float) and np.isnan(element))]
arrival = list(set(arrival))
arrival

["jusqu'à la",
 'mettre le cap sur ',
 'vers',
 'à',
 'prendre la direction de',
 'pour',
 'me rendre à',
 'à la',
 'vers la',
 'se diriger en direction de ',
 'à la gare de',
 "jusqu'à"]

In [906]:
depart = df['Departure words']
depart = list(depart)
depart = [element.lower() for element in depart]
depart = list(set(depart))
depart

['se déplacer vers',
 'allant de ',
 'je file vers la',
 'se déplacer de ',
 'aller de la',
 'de',
 'depuis la',
 'partir de',
 'cheminer à',
 'faire un trajet vers',
 'se rendre à',
 'faire un trajet de la',
 'cheminer de',
 'voyager vers',
 'voyager de ',
 'partir à',
 'se diriger vers ',
 'cheminer pour',
 'de la gare de',
 'de la',
 'faire un trajet de',
 'depuis ',
 'aller à',
 'se rendre de la',
 'depuis la ville de',
 'voyager à',
 'se rendre vers',
 'en partant de',
 'partir vers']

In [907]:
phrase = list(data['PHRASE'])
phrase = [element.lower() for element in phrase]
phrase = list(set(phrase))

In [934]:
import spacy

nlp = spacy.load("fr_core_news_sm")  # Charger le modèle SpaCy en français

phrase = "Si l'on projette un voyage depuis la ville alsacienne de Strasbourg vers la ville occitane de Toulouse, pourriez-vous nous guider dans la mise en place de cette aventure multimodale ?"

# Appliquer le modèle SpaCy à la phrase
doc = nlp(phrase)

# Détection des entités nommées
for entite in doc.ents:
    # Vérifier si l'entité est une ville (ou un lieu, selon le contexte)
    if entite.label_ == "LOC":
        print(f"Entité: {entite.text}, Label: {entite.label_}")


Entité: Strasbourg, Label: LOC
Entité: Toulouse, Label: LOC


In [972]:
import spacy
loc = []
# Load the spaCy language model
nlp = spacy.load("fr_core_news_sm")  # You may need to download the model using: python -m spacy download en_core_web_sm

# Define a phrase containing locations
phrase = "Si l'on projette un voyage depuis la gare de Strasbourg vers la ville occitane de Toulouse, pourriez-vous nous guider dans la mise en place de cette aventure multimodale ?"

# Process the phrase with spaCy
doc = nlp(phrase)

# Extract entities and their labels (including locations)
entities = [(ent.text, ent.label_) for ent in doc.ents]
#print(entities)
# Filter entities to get only locations
for j in range(0, len(entities)):
    print(entities[j][0])
#locations = [entity[0] for entity in entities if entity[1] == 'GPE' or entity[1] == 'LOC']

# Print the detected locations
print("Detected locations:", locations)

gare de Strasbourg
Toulouse
Detected locations: ['gare de Strasbourg', 'Toulouse']


In [967]:
import spacy

nlp = spacy.load("fr_core_news_sm")  # Charger le modèle SpaCy en français

phrase = "Je veux aller à voir mon ami Albert à Tours en partant de la gare de Bordeaux."

# Appliquer le modèle SpaCy à la phrase
doc = nlp(phrase)

trouver_vers = False  # Indicateur pour commencer la recherche après "vers"
locations = []  # Liste pour stocker les villes

# Parcourir les tokens (mots) dans le texte
for token in doc:
    for l in range(0, len(arrival)):
        if trouver_vers and token.ent_type_ == "LOC":  # Vérifier si l'entité est une localité
            locations.append(token.text)  # Ajouter la localité à la liste des locations
        elif token.text == arrival[l]:
            arr = arrival[l]
            trouver_vers = True  # Activer la recherche des mots suivant "vers"

# Imprimer la ville après "vers" (si elle existe)
if len(locations) > 0:
    print(f"La ville après {arr} est : {locations[0]}")
else:
    print("Aucune ville trouvée après 'vers'")

La ville après à est : Tours


In [946]:
doc = nlp(phrase)
for i in range(0, len(doc)):
    for l in range(0, len(arrival)):
        if doc[i].lower_ == arrival[l]:
            for k in range(0, len(locations)):
                if doc[i+1].lower_ == locations[k].lower():
                    print("arrivée", doc[i], locations[k])
    for j in range(0, len(depart)):
        if doc[i].lower_ == depart[j]:
            for k in range(0, len(locations)):
                if doc[i+1].lower_ == locations[k].lower():
                    print("depart", doc[i], locations[k])
                    break;
    i+=1
    #for j in range(0, len(locations)):
     #   if locations[j].lower() == doc[i].lower_:
      #      print(doc[i-1])

depart de Strasbourg
depart de Toulouse


In [837]:
gares = []

for k in range(0, len(locations)):
    if len(nlp(locations[k])) == 1:
        split_phrase = locations[k].split(" ")[0]
    elif len(len(locations[k])) == 2:
        split_phrase = locations[k].split(" ")[1]
    else:
        split_phrase = locations[k].split(" ")[2]

    if '-' in split_phrase:
        split_ville = split_phrase.split("-")[0]
    else:
        split_ville = split_phrase
    gares.append(split_ville)
gares

['Strasbourg', 'Toulouse']

In [823]:
doc = nlp(phrase)

In [849]:
for i in range(0, len(doc)):
    #print(doc[i])
    for j in range(0, len(gares)):
        if doc[i].lower_ == gares[j].lower():
            one_word = str(doc[i-1].lower_)
            two_words = str(doc[i-2].lower_)+' '+str(doc[i-1].lower_)
            third_words = str(doc[i-3].lower_)+' '+str(doc[i-2].lower_)+' '+str(doc[i-1].lower_)
            four_words = str(doc[i-4].lower_)+' '+str(doc[i-3].lower_)+' '+str(doc[i-2].lower_)+' '+str(doc[i-1].lower_)

            if four_words in arrival:
                print(doc[i], '4 arrival:')
            elif third_words in arrival:
                print(doc[i], '3 arrival:')
            elif two_words in arrival:
                print(doc[i], '2 arrival:')
            elif one_word in arrival:
                print(doc[i], '1 arrival:')
            
            elif four_words in depart:
                print(doc[i], '4 depart:')
            elif third_words in depart:
                print(doc[i], '3 depart:')
            elif two_words in depart:
                print(doc[i], '2 depart:')
            elif one_word in depart:
                print(doc[i], '1 depart:')

Strasbourg 1 depart:
Toulouse 1 depart:


In [659]:
v = []

for i in range(0, len(villes_gares)):
    doc = nlp(villes_gares[i])
    entities = [entity.text for entity in doc.ents]
    if len(entities)>0:
        v.append(entities)

In [681]:
villes_gares

['villebernier',
 'blemerey-54',
 'melicocq',
 'antran',
 'lorrez-le-bocage-preaux',
 'bazainville',
 'huisnes-sur-mer',
 'hendecourt-les-ransart',
 'fontette',
 'cauzac',
 'bonsmoulins',
 'beceleuf',
 'saint-gerand',
 'saint-andre-et-appelles',
 'notre-dame-d-oe',
 'saint-quentin-la-poterie',
 'chevroliere',
 'neuville-coppegueule',
 'charavines',
 'bourdet',
 'estenos',
 'gast',
 'jonchery-sur-suippe',
 'clavieres',
 'vexaincourt',
 'dombrot-le-sec',
 'sarp',
 'etais-la-sauvin',
 'houvin-houvigneul',
 'rosieres-en-santerre',
 'croisy',
 'melicourt',
 'grattery',
 'sens-sur-seille',
 'buzancy-08',
 'bouniagues',
 'saint-jouin-de-marnes',
 'meyrieu-les-etangs',
 'ivry-la-bataille',
 'saint-jean-de-barrou',
 'cherac',
 'rosieres-43',
 'saint-benoît-01',
 'siran-34',
 'blargies',
 'mosnay',
 'saint-mesmin-24',
 'm\x9coernach',
 'sant-antonino',
 'maison-maugis',
 'chomerac',
 'bosse-60',
 'courry',
 'ruhans',
 'chateaudouble-83',
 'baconniere',
 'saint-cirq-82',
 'mesnil-jourdain',
 'neu

In [679]:
len(v)

11338

In [700]:
for i in range(0, len(villes)):
    if villes[i] == "deaux".lower():
        print("OK")

OK


de
à
à


In [646]:
listt = []

def is_depart(doc):
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    for entity, label in entities:
        for i in range(0, len(villes_gares)):
            if villes_gares[i] in entity.lower():
                print(villes_gares[i])

    for i in range(0, len(doc)):
        
        if (doc[i].lower_ in villes_gares):
            if ((doc[i].pos_ == "PROPN" or doc[i].pos_ == "NOUN")):
                if (doc[i-1].pos_ == "ADP"):
                    two_words = str(doc[i-2].lower_)+' '+str(doc[i-1].lower_)
                    third_words = str(doc[i-3].lower_)+' '+str(doc[i-2].lower_)+' '+str(doc[i-1].lower_)
                    four_words = str(doc[i-4].lower_)+' '+str(doc[i-3].lower_)+' '+str(doc[i-2].lower_)+' '+str(doc[i-1].lower_)

                    if four_words in depart:
                        listt.append([doc, doc[i]])
                        return True
                    #elif third_words in depart:
                      #  listt.append([doc, doc[i]])
                    #elif two_words in depart:
                     #   listt.append([doc, doc[i]])
                    #if doc[i-1].lower_ in depart:
                        #listt.append([doc, doc[i]])
                        #return True

                    #if (doc[i-1].lower_ in depart or two_words in depart or  third_words in depart or four_words in depart):
                        #return True
    return False

for doc in nlp.pipe(["un train de la gare de lille flandres à la gare de bordeaux."]):
    #is_depart(doc)
    if is_depart(doc):
        _

listt

lille
deaux
bo
bordeaux


[]

In [505]:
k = nlp("J'aimerais des informations sur la manière de prendre un train de la Gare de Lille Flandres à la Gare de Bordeaux Saint-Jean, pouvez-vous me guider")

In [503]:
Quelle est la marche à suivre pour prendre un train de la Gare de Toulouse-Matabiau pour me rendre à la Gare de Nantes ? J'apprécierais vos conseils.k = nlp("")

In [506]:
tokens = [token.text for token in k]
tokens

["J'",
 'aimerais',
 'des',
 'informations',
 'sur',
 'la',
 'manière',
 'de',
 'prendre',
 'un',
 'train',
 'de',
 'la',
 'Gare',
 'de',
 'Lille',
 'Flandres',
 'à',
 'la',
 'Gare',
 'de',
 'Bordeaux',
 'Saint-Jean',
 ',',
 'pouvez',
 '-vous',
 'me',
 'guider']

In [531]:
pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [537]:
from geopy.distance import lonlat, distance
newport_ri_xy = (-71.312796, 41.49008)
cleveland_oh_xy = (-81.695391, 41.499498)
print(distance(lonlat(*newport_ri_xy), lonlat(*cleveland_oh_xy)).miles)

538.3904453677204
